In [1]:
import uuid
import asyncio  # Needed for asynchronous operations
from pathlib import Path

# Import Pyrit functions and classes for initialization, target setup, evaluation, and report generation.
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, HTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.common.text_helper import save_html_report, generate_single_turn_html_report
from pyrit.prompt_target.http_target.parsers.AHAssistantResponseParser import AHAssistantResponseParser

# ------------------------------------------------------------------------------
# Setup: Define Member ID, URL, and HTTP Request Template
# ------------------------------------------------------------------------------

# Member ID used to build the service URL
member_id = "16611078"

# Construct the URL for the assistant service using the member_id
url = f"<URL>{member_id}"

# Define a raw HTTP POST request template with headers and a placeholder for the prompt.
# Replace <YOUR_AUTH_TOKEN> with your actual token if needed.
start_chat_request_raw = f"""
    POST {url}
    Content-Type: application/json
    X-Authorization: <JWT_TOKEN>
    Accept: text/event-stream
    x-test-mode: true

    {{
        "data": "{{PROMPT}}"
    }}
"""

# ------------------------------------------------------------------------------
# Initialize the Pyrit Environment
# ------------------------------------------------------------------------------
# Use an in-memory database for a clean testing environment.
initialize_pyrit(memory_db_type=IN_MEMORY)

# ------------------------------------------------------------------------------
# Define QA Pairs for Testing
# ------------------------------------------------------------------------------
# Each QA pair contains a question and its expected answer suggestion chips.
qa_pairs = [
    {
        "question": "Ik zoek een scheermesje.",
        "answer": "SUGGESTION_CHIPS: [Wegwerpscheermesjes, Scheermesjes voor gevoelige huid, Scheerschuim, Scheermesjes schoonmaken]"
    }
]

# ------------------------------------------------------------------------------
# Setup: HTTP Target, Evaluator, and Orchestrator
# ------------------------------------------------------------------------------

# Create an HTTP target that sends prompts using the defined request template.
http_prompt_target = HTTPTarget(
    http_request=start_chat_request_raw,
    prompt_regex_string="{PROMPT}",
    timeout=60.0,
    callback_function=AHAssistantResponseParser.parse_response
)

# Create an evaluator that uses a YAML configuration for scoring suggestions.
scorer = Evaluator(
    chat_target=OpenAIChatTarget(),
    evaluator_yaml_path=Path("assets/AH_Evaluators/suggestionPillsEvaluator.yaml"),
    scorer_type="float_scale"
)

# Create the orchestrator for sending prompts and evaluating responses.
orchestrator = PromptSendingOrchestrator(
    objective_target=http_prompt_target,
    scorers=[scorer]
)

# ------------------------------------------------------------------------------
# Main Asynchronous Function: Send Prompts and Generate Report
# ------------------------------------------------------------------------------
async def main():
    # Extract the list of questions from the QA pairs.
    questions = [pair["question"] for pair in qa_pairs]
    
    # Send the list of prompts asynchronously.
    await orchestrator.send_prompts_async(prompt_list=questions)
    
    # Retrieve the chat results from the orchestrator.
    results = orchestrator.get_chat_results()  # Optionally, a threshold can be passed here.
    
    # ------------------------------------------------------------------------------
    # Generate and Save HTML Report
    # ------------------------------------------------------------------------------
    # Define the report directory path and create it if it doesn't exist.
    report_dir = Path("tests/E2E/reports/DataSet").resolve()
    report_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate and save an HTML report based on the chat results.
    save_html_report(
        results=results,
        is_chat_evaluation=False,
        report_generator=generate_single_turn_html_report,
        directory=str(report_dir)
    )

# ------------------------------------------------------------------------------
# Entry Point: Run the Main Asynchronous Function
# ------------------------------------------------------------------------------
await main()
